### Hands-on 5: 
Let's continue to use the dataset from Lecture 5 (from the wine magazine) to practice data transformation, grouping, and sorting.

In [1]:
import pandas as pd
import numpy as np
csvurl="https://gist.githubusercontent.com/clairehq/" + \
        "79acab35be50eaf1c383948ed3fd1129/raw/407a02139ae1e134992b90b4b2b8c329b3d73a6a/winemag-data-130k-v2.csv"
wine = pd.read_csv(csvurl)
wine

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65494,65494,France,Made from young vines from the Vaulorent porti...,Fourchaume Premier Cru,90,45.0,Burgundy,Chablis,NaN,Roger Voss,@vossroger,William Fèvre 2005 Fourchaume Premier Cru (Ch...,Chardonnay,William Fèvre
65495,65495,Australia,"This is a big, fat, almost sweet-tasting Caber...",NaN,90,22.0,South Australia,McLaren Vale,NaN,Joe Czerwinski,@JoeCz,Tapestry 2005 Cabernet Sauvignon (McLaren Vale),Cabernet Sauvignon,Tapestry
65496,65496,US,"Much improved over the unripe 2005, Fritz's 20...",Estate,90,20.0,California,Dry Creek Valley,Sonoma,NaN,NaN,Fritz 2006 Estate Sauvignon Blanc (Dry Creek V...,Sauvignon Blanc,Fritz
65497,65497,US,This wine wears its 15.8% alcohol better than ...,Block 24,90,31.0,California,Napa Valley,Napa,NaN,NaN,Hendry 2004 Block 24 Primitivo (Napa Valley),Primitivo,Hendry


#### Question 1: ####  
What is the mean of the points column?

In [2]:
wine_mean = wine.points.mean()
wine_mean

88.43403716087269

#### Question 2: ####  
How many countries are present in this dataset? (Only count each country once)

In [3]:
unique_countries = wine.country.nunique()
unique_countries

41

#### Question 3: ####
How many times does each country appeared in this dataset? Show each country and the corresponding count (show counts in ascending order)

In [4]:
country_count = wine.country.value_counts(ascending= True)
country_count

Slovakia                      1
Armenia                       1
Bosnia and Herzegovina        1
Switzerland                   4
India                         4
Luxembourg                    4
Ukraine                       5
Cyprus                        6
Czech Republic                6
Macedonia                     6
Serbia                        7
Peru                          8
Morocco                      11
Lebanon                      20
Moldova                      30
Mexico                       31
Brazil                       31
England                      36
Georgia                      37
Slovenia                     39
Turkey                       43
Croatia                      44
Uruguay                      61
Hungary                      61
Romania                      67
Bulgaria                     68
Canada                      108
Greece                      242
Israel                      259
New Zealand                 733
South Africa                737
Germany 

#### Question 4: ####
Create a variable `adjusted_price` containing the adjusted price which is the price subtracted by the average price. *This is called **"centering" transformation** - a method commonly used in the preprocessing step before applying various machine learning algorithms.*

In [5]:
adjusted_price = wine.price.map(lambda p: p - wine_mean)
adjusted_price

0              NaN
1       -73.434037
2       -74.434037
3       -75.434037
4       -23.434037
           ...    
65494   -43.434037
65495   -66.434037
65496   -68.434037
65497   -57.434037
65498   -78.434037
Name: price, Length: 65499, dtype: float64

#### Question 5: ####
What is the title of the wine that has the highest points-to-price ratio in the dataset?

In [6]:
ratio = wine['points']/wine['price']
max_id = ratio.idxmax()
wine.iloc[max_id]['title']

'Bandit NV Merlot (California)'

#### Question 6: ####
Create a series `flavor_counts` that contains two values: the number of wines that has the word "tart" in the `description` column and the number of wines that has the word "berries" in the `description` column. The index of the Series should be "Tart" and "Berries" for the corresponding values.

In [7]:
tart_count = wine.description.str.contains(" tart").sum()
berries_count = wine.description.str.contains(" berries").sum()
#tart_count
#berries_count
new_index = ['Tart','Berries']
count = [tart_count, berries_count]
flavor_counts = pd.Series(data = count, index = new_index)
print(flavor_counts)


Tart       3058
Berries    1178
dtype: int64


#### Question 7: ####
Let's convert the points into simple star ratings. A score of 90 or higher counts as 3 stars, a score of at least 80 but less than 90 is 2 stars. Any other score is 1 star.

Also, any wines from France should automatically get 3 stars, regardless of points.

Add this new column `star_ratings` to the dataframe with the number of stars for each wine in the dataset. 

In [8]:
condition1 = [
    (wine['points'] >= 90) | (wine['country'] == 'France'),
    (wine['points'] >= 80) & (wine['points'] < 90) & (wine['country'] != 'France'),
    (wine['points'] > 80) & (wine['country'] != 'France')
    ]

#condition2 = [wine['country'] == 'France']
values1 = ['3', '2', '1']
#values2 = ['3']
wine['star_ratings'] = np.select(condition1, values1)
wine

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,star_ratings
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65494,65494,France,Made from young vines from the Vaulorent porti...,Fourchaume Premier Cru,90,45.0,Burgundy,Chablis,NaN,Roger Voss,@vossroger,William Fèvre 2005 Fourchaume Premier Cru (Ch...,Chardonnay,William Fèvre,3
65495,65495,Australia,"This is a big, fat, almost sweet-tasting Caber...",NaN,90,22.0,South Australia,McLaren Vale,NaN,Joe Czerwinski,@JoeCz,Tapestry 2005 Cabernet Sauvignon (McLaren Vale),Cabernet Sauvignon,Tapestry,3
65496,65496,US,"Much improved over the unripe 2005, Fritz's 20...",Estate,90,20.0,California,Dry Creek Valley,Sonoma,NaN,NaN,Fritz 2006 Estate Sauvignon Blanc (Dry Creek V...,Sauvignon Blanc,Fritz,3
65497,65497,US,This wine wears its 15.8% alcohol better than ...,Block 24,90,31.0,California,Napa Valley,Napa,NaN,NaN,Hendry 2004 Block 24 Primitivo (Napa Valley),Primitivo,Hendry,3


#### Question 8: ####
Who are the most common wine reviewers in the dataset? Create a Series whose index is the taster_twitter_handle category from the dataset, and whose values count how many reviews each person wrote.

In [9]:
reviewer = wine['taster_twitter_handle'].value_counts(ascending = False)
reviewer

@vossroger          13045
@wineschach          7752
@kerinokeefe         5313
@paulgwine           4851
@vboone              4696
@mattkettmann        3035
@JoeCz               2605
@wawinereport        2358
@gordone_cellars     2032
@AnneInVino          1769
@laurbuzz             938
@suskostrzewa         593
@worldwineguys        465
@bkfiona               11
@winewchristina         4
Name: taster_twitter_handle, dtype: int64

#### Question 9: ####
What combination of countries and varieties are most common? Create a Series whose index is a MultiIndexof {country, variety} pairs. For example, a pinot noir produced in the US should map to {"US", "Pinot Noir"}. Sort the values in the Series in descending order based on wine count.

In [12]:
#countries_wine = wine.groupby(['country', 'variety']).description.agg([len]).sort_values(by = 'len', ascending = False)
countries_wine = wine.groupby(['country', 'variety'])['description'].count().sort_values(ascending = False)
countries_wine

country    variety                 
US         Pinot Noir                  4918
           Cabernet Sauvignon          3649
           Chardonnay                  3412
France     Bordeaux-style Red Blend    2380
Italy      Red Blend                   1870
                                       ... 
           Groppello                      1
           Grignolino                     1
           Gragnano                       1
           Gaglioppo                      1
Argentina  Barbera                        1
Name: description, Length: 1304, dtype: int64

#### Question 10 #####
Create a Series whose index is reviewers and whose values is the average score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [11]:
reviewers_and_avg_score = wine.groupby(['taster_name']).points.mean()
reviewers_and_avg_score

taster_name
Alexander Peartree    86.014286
Anna Lee C. Iijima    88.380506
Anne Krebiehl MW      90.587903
Carrie Dykes          86.644444
Christina Pickard     89.500000
Fiona Adams           87.090909
Jeff Jenssen          88.273504
Jim Gordon            88.604331
Joe Czerwinski        88.519770
Kerin O’Keefe         88.827969
Lauren Buzzeo         87.831557
Matt Kettmann         90.021087
Michael Schachner     86.904541
Mike DeSimone         89.030303
Paul Gregutt          89.095032
Roger Voss            88.678957
Sean P. Sullivan      88.666243
Susan Kostrzewa       86.408094
Virginie Boone        89.229557
Name: points, dtype: float64